# Khám phá dữ liệu (đan xen với pha tiền xử lý dữ liệu)

### Đọc dữ liệu từ tập tin 

Đọc dữ liệu từ file `score_college_exam_data.csv` và lưu vào DataFrame `df`.

In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv('../data/score_college_exam_data.csv', index_col= "Unnamed: 0")
df.head()

,Year,sbd,Toan,Van,Ngoai_ngu,Li,Hoa,Sinh,Su,Dia,GDCD
0,2019,24008611,8.2,8.00,7.6,NaN,NaN,NaN,8.25,7.75,8.75
1,2019,51000032,8.6,6.17,8.0,8.25,4.5,6.0,NaN,NaN,NaN
2,2019,51000005,5.2,4.75,NaN,NaN,NaN,NaN,4.00,4.25,5.75
3,2019,51000021,4.4,4.50,NaN,NaN,NaN,NaN,2.25,4.75,5.00
4,2019,51000013,5.8,6.00,4.4,NaN,NaN,NaN,7.75,8.25,7.50


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

Ta tính số dòng và số cột của DataFrame `core_df` và lưu vào biến `shape` (tuple). 

In [14]:
shape = df.shape
shape

(3665169, 11)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng trong DataFrame `core_df` cho biết kết quả điểm thi đại học của học sinh Việt Nam từ năm 2019 đến năm 2022. Có vẻ không có vấn đề các dòng có ý nghĩa khác nhau (lúc sau, nếu phát hiện ra vấn đề thì ta sẽ quay lại đây). 

### Dữ liệu có các dòng bị lặp không? Nếu có thì ta phải drop dòng trùng và ghi lại file.

In [15]:
num_duplicated_rows = df.duplicated().sum()
if(num_duplicated_rows == 0):
    print("Dữ liệu không bị lặp")
else: 
    print(f"Dữ liệu bị lặp {num_duplicated_rows} dòng")
    df = df.drop_duplicates()
    df.to_csv('../data/score_college_exam_data.csv')

Dữ liệu bị lặp 4591 dòng


Kiểm tra lại dữ liệu

In [16]:
if num_duplicated_rows != 0:
    num_duplicated_rows = df.duplicated().sum()
    if(num_duplicated_rows == 0):
        print("Dữ liệu không bị lặp")
    else: 
        print(f"Dữ liệu bị lặp {num_duplicated_rows} dòng")

Dữ liệu không bị lặp


## Mỗi cột có ý nghĩa gì?

- **Year** là năm thi của thí sinh

- **sbd** là số báo danh của thí sinh

- Các cột còn lại là điểm thi theo các môn của các thí sinh. Những giá trị NaN là môn mà thí sinh đó không tham gia thi.

In [17]:
df.columns

Index(['Year', 'sbd', 'Toan', 'Van', 'Ngoai_ngu', 'Li', 'Hoa', 'Sinh', 'Su',
       'Dia', 'GDCD'],
      dtype='object')


### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3660578 entries, 0 to 995440
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Year       int64  
 1   sbd        int64  
 2   Toan       float64
 3   Van        float64
 4   Ngoai_ngu  float64
 5   Li         float64
 6   Hoa        float64
 7   Sinh       float64
 8   Su         float64
 9   Dia        float64
 10  GDCD       float64
dtypes: float64(9), int64(2)
memory usage: 335.1 MB


Các cột đều có dạng `numeric` và đều phù hợp nên không cần xử lý kiểu dữ liệu của các cột nữa.

### Với mỗi cột có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

Với các cột điểm, Ta sẽ tính:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu 
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max


Bạn sẽ lưu kết quả vào DataFrame `num_col_info_df`, trong đó: 
- Tên của các cột là tên của các cột điểm trong `core_df`
- Tên của các dòng là: "missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"

Để dễ nhìn, tất cả các giá trị ta đều làm tròn với 1 chữ số thập phân bằng phương thức `.round(1)`.

In [19]:
def get(col):
    NaNValue = col.isna().sum()
    valu1 = col.dropna()
    x = []
    x.append((NaNValue/len(col))*100)
    x.append(np.percentile(valu1, 0))
    x.append(np.percentile(valu1, 25))
    x.append(np.percentile(valu1, 50))
    x.append(np.percentile(valu1, 75))
    x.append(np.percentile(valu1, 100))
    return np.array(x).round(1)

index_info=["missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"]
dict_info = {}
for i in df.columns:
    if df[i].dtype in [np.float32, np.float64]:
        dict_info[i] = get(df[i])   
num_col_info_df = pd.DataFrame(dict_info,index_info)
num_col_info_df

,Toan,Van,Ngoai_ngu,Li,Hoa,Sinh,Su,Dia,GDCD
missing_ratio,0.8,1.5,11.6,65.7,65.5,66.0,34.4,35.1,44.3
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lower_quartile,5.2,5.5,3.4,5.5,5.2,4.2,3.8,5.8,7.2
median,6.6,6.5,4.6,6.8,6.5,5.0,5.0,6.8,8.2
upper_quartile,7.8,7.2,6.4,7.5,7.8,6.0,6.5,7.5,9.0
max,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
